In [1]:
import numpy as np
import os
import pandas as pd
import sys
sys.path.append("..")

from sklearn import metrics

In [148]:
loc = "../data/categorical/"
dirs = os.listdir(loc)
pns = sorted([d for d in dirs if "_results.pickle" in d])

RES_AUC = {}
for pn in pns:
    df = pd.read_pickle(loc + pn)
    res_AUC = []
    for trial in range(100):
        pred = df.iloc[trial][0]
        true = df.iloc[trial][1]
        res_AUC.append(metrics.roc_auc_score(true, pred))
    RES_AUC[pn[:-15]] = res_AUC
    print(pn)

Led_NB_results.pickle
Led_RF_results.pickle
Led_SVM_results.pickle
PhosS_NB_results.pickle
PhosS_RF_results.pickle
PhosS_SVM_results.pickle
boundary_NB_results.pickle
boundary_RF_results.pickle
boundary_SVM_results.pickle
cam_NB_results.pickle
cam_RF_results.pickle
cam_SVM_results.pickle
car-good_NB_results.pickle
car-good_RF_results.pickle
car-good_SVM_results.pickle


In [149]:
categ_AUC = pd.DataFrame(RES_AUC)
categ_AUC.shape

(100, 15)

In [160]:
conti_AUC.iloc[:, 1::3]

,compustat_RF,glass4_RF,mammography_RF,oil_RF,winequality-red8vs6_RF,yeast-2vs8_RF
0,0.747433,0.837341,0.889661,0.584449,0.482759,0.647835
1,0.725569,0.756011,0.866218,0.498884,0.492163,0.517965
2,0.742445,0.775813,0.861823,0.660342,0.544584,0.650000
3,0.733382,0.837341,0.875560,0.613467,0.492163,0.489177
4,0.734544,0.628006,0.881973,0.679315,0.641588,0.793506
...,...,...,...,...,...,...
95,0.718383,0.898868,0.847628,0.584449,0.550853,0.452381
96,0.765918,0.608204,0.880692,0.702381,0.553988,0.528355
97,0.742089,0.827440,0.861914,0.645461,0.498433,0.597835
98,0.747793,0.751061,0.868328,0.560268,0.641588,0.489177


In [167]:
print(round(mixed_AUC.iloc[:, 0::3].mean().mean(), 3))
print(round(mixed_AUC.iloc[:, 1::3].mean().mean(), 3))
print(round(mixed_AUC.iloc[:, 2::3].mean().mean(), 3))

0.707
0.834
0.815


In [152]:
auc = pd.concat([conti_AUC, categ_AUC, mixed_AUC], axis=1)
auc.head(3)

,compustat_NB,compustat_RF,compustat_SVM,glass4_NB,glass4_RF,glass4_SVM,mammography_NB,mammography_RF,mammography_SVM,oil_NB,...,flare-F_SVM,hypo_NB,hypo_RF,hypo_SVM,sick_NB,sick_RF,sick_SVM,vowel0_NB,vowel0_RF,vowel0_SVM
0,0.597959,0.747433,0.730189,0.832390,0.837341,0.485149,0.867973,0.889661,0.902386,0.639881,...,0.722212,0.878414,0.903644,0.933348,0.686604,0.783684,0.662364,0.812051,0.965479,0.981069
1,0.504137,0.725569,0.720702,0.770863,0.756011,0.475248,0.856530,0.866218,0.892222,0.588542,...,0.673562,0.807912,0.949741,0.951772,0.671858,0.802561,0.728045,0.783197,0.944395,0.987751
2,0.528534,0.742445,0.738832,0.785714,0.775813,0.485149,0.875212,0.861823,0.905318,0.726190,...,0.648615,0.857244,0.936945,0.940003,0.657747,0.781925,0.669872,0.728780,0.884410,0.985523


In [169]:
auc.to_csv("../data/RES_AUC.csv", index=False)

In [58]:
RES_F = {}
RES_G = {}
for pn in pns:
    df = pd.read_pickle(loc + pn)
    res_F = []
    res_G = []
    for trial in range(100):
        pred = df.iloc[trial][0]
        true = df.iloc[trial][1]
        table = pd.crosstab(index=true, columns=pred, rownames=["Actual"], colnames=["Assigned"])
        TN = table.loc[0,0]
        FN = table.loc[1,0]

        if table.shape == (2,2):
            TP = table.loc[1,1]
            FP = table.loc[0,1]

            Rec = TP/(TP+FN)
            Pre = TP/(TP+FP)
            Spe = TN/(TN+FN)
    
            F1 = 2*Rec*Pre/(Rec+Pre)
            Gm = np.sqrt(Rec * Spe)
        else:
            F1 = 0
            Gm = 0
            
        res_F.append(F1)
        res_G.append(Gm)
    RES_F[pn[:-15]] = res_F
    RES_G[pn[:-15]] = res_G

1. majority(0) = negative
2. minority(1) = positive

Recall = TP / (TP + FN)

Precision = TP / (TP + FP)

Specificity = 𝑇𝑁/ (𝑇𝑁 + 𝐹N)

* TP (True Positive) - The positive instances correctly classified.
* FN (False Negative) – The negative instances incorrectly classified.

In [59]:
RES_F_df = pd.DataFrame(RES_F)
RES_F_df.columns = [c+"_F1" for c in RES_F_df.columns]

RES_G_df = pd.DataFrame(RES_G)
RES_G_df.columns = [c+"_Gmean" for c in RES_G_df.columns]

RES_F_df.shape, RES_G_df.shape

((100, 15), (100, 15))

In [60]:
RES = pd.concat([RES_F_df, RES_G_df], axis=1)
RES.head(3)

,abalone_NB_F1,abalone_RF_F1,abalone_SVM_F1,flare-F_NB_F1,flare-F_RF_F1,flare-F_SVM_F1,hypo_NB_F1,hypo_RF_F1,hypo_SVM_F1,sick_NB_F1,...,flare-F_SVM_Gmean,hypo_NB_Gmean,hypo_RF_Gmean,hypo_SVM_Gmean,sick_NB_Gmean,sick_RF_Gmean,sick_SVM_Gmean,vowel0_NB_Gmean,vowel0_RF_Gmean,vowel0_SVM_Gmean
0,0.104000,0.119522,0.247788,0.121864,0.283186,0.236364,0.475336,0.728571,0.733333,0.183124,...,0.760911,0.927837,0.909566,0.946528,0.943608,0.884708,0.881220,0.870537,1.000000,1.0
1,0.068182,0.090435,0.238994,0.161702,0.287879,0.229885,0.266355,0.750000,0.703030,0.177419,...,0.665519,0.963754,0.964605,0.973526,0.927861,0.905534,0.893739,0.815629,0.962731,1.0
2,0.100279,0.100877,0.214286,0.275000,0.226804,0.250000,0.343373,0.769231,0.727273,0.168317,...,0.594381,0.964120,0.946550,0.955591,0.942714,0.889823,0.779235,0.727607,0.885316,1.0


In [61]:
RES = RES[sorted(RES.columns)]
RES.shape

(100, 30)

In [62]:
RES.head(3)

,abalone_NB_F1,abalone_NB_Gmean,abalone_RF_F1,abalone_RF_Gmean,abalone_SVM_F1,abalone_SVM_Gmean,flare-F_NB_F1,flare-F_NB_Gmean,flare-F_RF_F1,flare-F_RF_Gmean,...,sick_RF_F1,sick_RF_Gmean,sick_SVM_F1,sick_SVM_Gmean,vowel0_NB_F1,vowel0_NB_Gmean,vowel0_RF_F1,vowel0_RF_Gmean,vowel0_SVM_F1,vowel0_SVM_Gmean
0,0.104000,0.663860,0.119522,0.713859,0.247788,0.689989,0.121864,0.871079,0.283186,0.847008,...,0.304054,0.884708,0.179283,0.881220,0.469799,0.870537,0.743802,1.000000,0.841121,1.0
1,0.068182,0.781345,0.090435,0.944582,0.238994,0.805523,0.161702,0.924968,0.287879,0.926027,...,0.317568,0.905534,0.226322,0.893739,0.473282,0.815629,0.785047,0.962731,0.891089,1.0
2,0.100279,0.782657,0.100877,0.886956,0.214286,0.714421,0.275000,0.698889,0.226804,0.698582,...,0.296417,0.889823,0.210682,0.779235,0.438596,0.727607,0.757895,0.885316,0.873786,1.0


In [63]:
RES.fillna(0, inplace=True)
RES.head(3)

,abalone_NB_F1,abalone_NB_Gmean,abalone_RF_F1,abalone_RF_Gmean,abalone_SVM_F1,abalone_SVM_Gmean,flare-F_NB_F1,flare-F_NB_Gmean,flare-F_RF_F1,flare-F_RF_Gmean,...,sick_RF_F1,sick_RF_Gmean,sick_SVM_F1,sick_SVM_Gmean,vowel0_NB_F1,vowel0_NB_Gmean,vowel0_RF_F1,vowel0_RF_Gmean,vowel0_SVM_F1,vowel0_SVM_Gmean
0,0.104000,0.663860,0.119522,0.713859,0.247788,0.689989,0.121864,0.871079,0.283186,0.847008,...,0.304054,0.884708,0.179283,0.881220,0.469799,0.870537,0.743802,1.000000,0.841121,1.0
1,0.068182,0.781345,0.090435,0.944582,0.238994,0.805523,0.161702,0.924968,0.287879,0.926027,...,0.317568,0.905534,0.226322,0.893739,0.473282,0.815629,0.785047,0.962731,0.891089,1.0
2,0.100279,0.782657,0.100877,0.886956,0.214286,0.714421,0.275000,0.698889,0.226804,0.698582,...,0.296417,0.889823,0.210682,0.779235,0.438596,0.727607,0.757895,0.885316,0.873786,1.0


In [64]:
RES.to_csv("../data/RES_mixed_exceptAUC.csv", index=False)

# Merge All

In [1]:
import numpy as np
import os
import pandas as pd
import sys
sys.path.append("..")

In [2]:
conti = "../data/RES_continuous_exceptAUC.csv"
categ = "../data/RES_categorical_exceptAUC.csv"
mixed = "../data/RES_mixed_exceptAUC.csv"

In [12]:
for fn in [conti, categ, mixed]:
    tmp = pd.read_csv(fn)
    print(round(tmp.mean(), 3))

compustat_NB_F1                  0.139
compustat_NB_Gmean               0.727
compustat_RF_F1                  0.206
compustat_RF_Gmean               0.818
compustat_SVM_F1                 0.183
compustat_SVM_Gmean              0.814
glass4_NB_F1                     0.504
glass4_NB_Gmean                  0.710
glass4_RF_F1                     0.579
glass4_RF_Gmean                  0.806
glass4_SVM_F1                    0.080
glass4_SVM_Gmean                 0.178
mammography_NB_F1                0.224
mammography_NB_Gmean             0.921
mammography_RF_F1                0.373
mammography_RF_Gmean             0.887
mammography_SVM_F1               0.522
mammography_SVM_Gmean            0.899
oil_NB_F1                        0.278
oil_NB_Gmean                     0.710
oil_RF_F1                        0.259
oil_RF_Gmean                     0.533
oil_SVM_F1                       0.114
oil_SVM_Gmean                    0.358
winequality-red8vs6_NB_F1        0.121
winequality-red8vs6_NB_Gm

In [81]:
types = ["continuous", "categorical", "mixed"]
for idx, fn in enumerate([conti, categ, mixed]):
    print(f"\n{types[idx]}")
    df = pd.read_csv(fn)
    for i in range(6):
        ave = df.mean()[i::6]
        print(f"{df.mean()[i::6].index[0][-6:]}: {ave.mean():,.3}")


continuous
_NB_F1: 0.279
_Gmean: 0.698
_RF_F1: 0.315
_Gmean: 0.635
SVM_F1: 0.26
_Gmean: 0.597

categorical
_NB_F1: 0.329
_Gmean: 0.716
_RF_F1: 0.312
_Gmean: 0.655
SVM_F1: 0.301
_Gmean: 0.442

mixed
_NB_F1: 0.229
_Gmean: 0.846
_RF_F1: 0.454
_Gmean: 0.888
SVM_F1: 0.466
_Gmean: 0.853


In [82]:
ave

abalone_SVM_Gmean    0.721454
flare-F_SVM_Gmean    0.717357
hypo_SVM_Gmean       0.964404
sick_SVM_Gmean       0.865285
vowel0_SVM_Gmean     0.998224
dtype: float64

In [94]:
types = ["continuous", "categorical", "mixed"]
RES = {}
for idx, fn in enumerate([conti, categ, mixed]):
    print(f"\n{types[idx]}")
    df = pd.read_csv(fn)
    for i in range(6):
        ave = df.mean()[i::6]
        RES[types[idx]+str(i)] = ave


continuous

categorical

mixed


In [95]:
RES.keys()

dict_keys(['continuous0', 'continuous1', 'continuous2', 'continuous3', 'continuous4', 'continuous5', 'categorical0', 'categorical1', 'categorical2', 'categorical3', 'categorical4', 'categorical5', 'mixed0', 'mixed1', 'mixed2', 'mixed3', 'mixed4', 'mixed5'])

In [101]:
RES[t + str(2)]

abalone_RF_F1    0.119167
flare-F_RF_F1    0.300731
hypo_RF_F1       0.760689
sick_RF_F1       0.298103
vowel0_RF_F1     0.789320
dtype: float64

In [102]:
NB_F1 = []
NB_GM = []
RF_F1 = []
RF_GM = []
SVM_F1 = []
SVM_GM = []
for t in types:
    NB_F1.append(RES[t + str(0)])
    NB_GM.append(RES[t + str(1)])
    RF_F1.append(RES[t + str(2)])
    RF_GM.append(RES[t + str(3)])
    SVM_F1.append(RES[t + str(4)])
    SVM_GM.append(RES[t + str(5)])

In [108]:
svmGmean = pd.concat(SVM_GM)
svmF1 = pd.concat(SVM_F1)
RFGmean = pd.concat(RF_GM)
RFF1 = pd.concat(RF_F1)
NBGmean = pd.concat(NB_GM)
NBF1 = pd.concat(NB_F1)

In [123]:
NB_idx = [i[:-6] for i in NBF1.index]
SVM_idx = [i[:-7] for i in svmF1.index]

In [124]:
NBF1

compustat_NB_F1              0.138629
glass4_NB_F1                 0.504167
mammography_NB_F1            0.223907
oil_NB_F1                    0.278259
winequality-red8vs6_NB_F1    0.120909
yeast-2vs8_NB_F1             0.409046
Led_NB_F1                    0.643241
PhosS_NB_F1                  0.183783
boundary_NB_F1               0.083473
cam_NB_F1                    0.239468
car-good_NB_F1               0.495346
abalone_NB_F1                0.087746
flare-F_NB_F1                0.243651
hypo_NB_F1                   0.251753
sick_NB_F1                   0.161923
vowel0_NB_F1                 0.398696
dtype: float64

In [125]:
svmGmean.index = SVM_idx
svmF1.index = SVM_idx
RFGmean.index = NB_idx
RFF1.index = NB_idx
NBGmean.index = NB_idx
NBF1.index = NB_idx

In [126]:
NBF1

compustat              0.138629
glass4                 0.504167
mammography            0.223907
oil                    0.278259
winequality-red8vs6    0.120909
yeast-2vs8             0.409046
Led                    0.643241
PhosS                  0.183783
boundary               0.083473
cam                    0.239468
car-good               0.495346
abalone                0.087746
flare-F                0.243651
hypo                   0.251753
sick                   0.161923
vowel0                 0.398696
dtype: float64

In [128]:
RES_ALL = pd.concat([svmF1, svmGmean, RFF1, RFGmean, NBF1, NBGmean], axis=1)
RES_ALL

,0,1,2,3,4,5
compustat,0.182877,0.813998,0.206180,0.818466,0.138629,0.727178
glass4,0.079568,0.177957,0.578956,0.805754,0.504167,0.709696
mammography,0.521983,0.899168,0.372922,0.887282,0.223907,0.920978
oil,0.114169,0.358069,0.258521,0.532564,0.278259,0.710411
winequality-red8vs6,0.105526,0.637815,0.126483,0.305419,0.120909,0.337699
yeast-2vs8,0.555145,0.695191,0.345898,0.458587,0.409046,0.783122
Led,0.748915,0.876793,0.650412,0.926143,0.643241,0.881152
PhosS,0.000647,0.005552,0.217628,0.507516,0.183783,0.642112
boundary,0.002837,0.011229,0.070664,0.240942,0.083473,0.340386
cam,0.172411,0.323802,0.176471,0.605324,0.239468,0.717462


In [129]:
RES_ALL.columns = ["svmF1", "svmGmean", "RFF1", "RFGmean", "NBF1", "NBGmean"]
RES_ALL

,svmF1,svmGmean,RFF1,RFGmean,NBF1,NBGmean
compustat,0.182877,0.813998,0.206180,0.818466,0.138629,0.727178
glass4,0.079568,0.177957,0.578956,0.805754,0.504167,0.709696
mammography,0.521983,0.899168,0.372922,0.887282,0.223907,0.920978
oil,0.114169,0.358069,0.258521,0.532564,0.278259,0.710411
winequality-red8vs6,0.105526,0.637815,0.126483,0.305419,0.120909,0.337699
yeast-2vs8,0.555145,0.695191,0.345898,0.458587,0.409046,0.783122
Led,0.748915,0.876793,0.650412,0.926143,0.643241,0.881152
PhosS,0.000647,0.005552,0.217628,0.507516,0.183783,0.642112
boundary,0.002837,0.011229,0.070664,0.240942,0.083473,0.340386
cam,0.172411,0.323802,0.176471,0.605324,0.239468,0.717462


In [133]:
RES_ALL = RES_ALL[["svmGmean", "svmF1", "RFGmean", "RFF1", "NBGmean", "NBF1"]]
RES_ALL

,svmGmean,svmF1,RFGmean,RFF1,NBGmean,NBF1
compustat,0.813998,0.182877,0.818466,0.206180,0.727178,0.138629
glass4,0.177957,0.079568,0.805754,0.578956,0.709696,0.504167
mammography,0.899168,0.521983,0.887282,0.372922,0.920978,0.223907
oil,0.358069,0.114169,0.532564,0.258521,0.710411,0.278259
winequality-red8vs6,0.637815,0.105526,0.305419,0.126483,0.337699,0.120909
yeast-2vs8,0.695191,0.555145,0.458587,0.345898,0.783122,0.409046
Led,0.876793,0.748915,0.926143,0.650412,0.881152,0.643241
PhosS,0.005552,0.000647,0.507516,0.217628,0.642112,0.183783
boundary,0.011229,0.002837,0.240942,0.070664,0.340386,0.083473
cam,0.323802,0.172411,0.605324,0.176471,0.717462,0.239468


In [134]:
commonIdx = [
    'boundary', 'cam', 'car-good', 'Led', 'PhosS', 'compustat',
    'glass4', 'mammography', 'oil', 'winequality-red8vs6',
    'yeast-2vs8', 'vowel0', 'sick', 'hypo',
    'flare-F', 'abalone'
]

In [176]:
auc_NB = auc.mean()[0::3]
auc_RF = auc.mean()[1::3]
auc_SVM = auc.mean()[2::3]

In [187]:
auc_SVM.index = SVM_idx
auc_NB.index = NB_idx
auc_RF.index = NB_idx

In [ ]:
auc_SVM.index = SVM_idx
svmF1.index = SVM_idx
RFGmean.index = NB_idx
RFF1.index = NB_idx
NBGmean.index = NB_idx
NBF1.index = NB_idx

In [189]:
AUC = pd.concat([auc_SVM, auc_RF, auc_NB], axis=1)
AUC.columns = ['SVM', 'RF', 'NB']
AUC.head(3)

,SVM,RF,NB
compustat,0.723587,0.742662,0.587791
glass4,0.548663,0.819173,0.755368
mammography,0.890682,0.866216,0.857110


In [191]:
AUC = AUC.loc[commonIdx]
AUC

,SVM,RF,NB
boundary,0.500563,0.521318,0.530603
cam,0.550755,0.613640,0.688983
car-good,0.964059,0.942459,0.955547
Led,0.878059,0.893645,0.861186
PhosS,0.500122,0.600640,0.622276
compustat,0.723587,0.742662,0.587791
glass4,0.548663,0.819173,0.755368
mammography,0.890682,0.866216,0.857110
oil,0.532455,0.629877,0.667065
winequality-red8vs6,0.620336,0.553495,0.561113


In [192]:
AUC.to_csv("../data/RES_AUC_sorted.csv", index=False)